# Homework

Download the model from here: 

https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/model_2024_hairstyle.keras

In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.lite as tflite
import numpy as np


### Question 1

Now convert this model from Keras to TF-Lite format.

What's the size of the **converted** model?

* 27 Mb
* 43 Mb
* `77 Mb`
* 127 Mb

In [2]:
model = keras.models.load_model('model_2024_hairstyle.keras')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('model_2024_hairstyle.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/km/dql4gckx6ps4lrl0mdrhnsbh0000gn/T/tmp9c5h6x_y/assets


INFO:tensorflow:Assets written to: /var/folders/km/dql4gckx6ps4lrl0mdrhnsbh0000gn/T/tmp9c5h6x_y/assets


Saved artifact at '/var/folders/km/dql4gckx6ps4lrl0mdrhnsbh0000gn/T/tmp9c5h6x_y'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  5956103888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5956104656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5956105424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5956106192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5956107344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5956107728: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1733457438.542060 4143624 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1733457438.542171 4143624 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2024-12-05 22:57:18.542674: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/km/dql4gckx6ps4lrl0mdrhnsbh0000gn/T/tmp9c5h6x_y
2024-12-05 22:57:18.542999: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-12-05 22:57:18.543004: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/km/dql4gckx6ps4lrl0mdrhnsbh0000gn/T/tmp9c5h6x_y
I0000 00:00:1733457438.545705 4143624 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2024-12-05 22:57:18.546202: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-12-05 22:57:18.612941: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /var/folder

In [3]:
import os
size = os.path.getsize('model_2024_hairstyle.tflite') // 1024 // 1024
print("The size of the converted model is:", size, 'Mb')

The size of the converted model is: 76 Mb


A/ `76 Mb`

### Question 2

To be able to use this model, we need to know the index of the input and 
the index of the output. 

What's the output index for this model?

* 3
* 7
* `13`
* 24


In [4]:
interpreter = tflite.Interpreter(model_path='model_2024_hairstyle.tflite')
interpreter.allocate_tensors()
output_index = interpreter.get_output_details()[0]['index']
print("The output index for this model is:", output_index)

The output index for this model is: 13


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


A/ `13`

### Preparing the image

You'll need some code for downloading and resizing images. You can use 
this code:

```python
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img
```

For that, you'll need to have `pillow` installed:

```bash
pip install pillow
```

Let's download and resize this image: 

https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

Based on the previous homework, what should be the target size for the image? R/ `(200, 200)`

In [17]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [18]:
img = download_image('https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg')
img = prepare_image(img, (200, 200))

### Question 3

Now we need to turn the image into numpy array and pre-process it. 

> Tip: Check the previous homework. What was the pre-processing 
> we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

* `0.24`
* 0.44
* 0.64
* 0.84

In [19]:
def preprocess_input(x):
    return x / 255.0

In [20]:
x = np.array(img, dtype='float32')
X = np.array([x])
X = preprocess_input(X)

In [21]:
X.shape

(1, 200, 200, 3)

In [22]:
print("The value in the first pixel is", round(X[0, 0, 0, 0], 2))

The value in the first pixel is 0.24


A/ `0.24`

### Question 4

Now let's apply this model to this image. What's the output of the model?

* 0.293
* 0.493
* 0.693
* `0.893`

In [23]:
input_index = interpreter.get_input_details()[0]['index']

interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
print("The output of the model is:", round(preds[0, 0], 4))

The output of the model is: 0.8938


A/ `0.8938`

### Prepare the lambda code 

Now you need to copy all the code into a separate python file. You will 
need to use this file for the next two questions.

Tip: you can test this file locally with `ipython` or Jupyter Notebook 
by importing the file and invoking the function from this file. 

In [41]:
import handler

event = { 'url': 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg' }

handler.handler(event, None)

np.float32(0.8937725)

### Docker 

For the next two questions, we'll use a Docker image that we already 
prepared. This is the Dockerfile that we used for creating the image:

```docker
FROM public.ecr.aws/lambda/python:3.10

COPY model_2024_hairstyle_v2.tflite .

RUN pip install numpy==1.23.1
```

Note that it uses Python 3.10. The latest models of TF Lite
do not support Python 3.12 yet, so we need to use 3.10. Also,
for this part, we will use TensorFlow 2.14.0. We have tested
it, and the models created with 2.17 could be served with 2.14.0.

For that image, we also needed to use an older version of numpy
(1.23.1)

The docker image is published to [`agrigorev/model-2024-hairstyle:v3`](https://hub.docker.com/r/agrigorev/model-2024-hairstyle/tags).

A few notes:

* The image already contains a model and it's not the same model
  as the one we used for questions 1-4.
* The wheel for this combination that you'll need to use in your Docker image is https://github.com/alexeygrigorev/tflite-aws-lambda/raw/main/tflite/tflite_runtime-2.14.0-cp310-cp310-linux_x86_64.whl


### Question 5

Download the base image `agrigorev/model-2024-hairstyle:v3`. You can do it with [`docker pull`](https://docs.docker.com/engine/reference/commandline/pull/).

So what's the size of this base image?

* 182 Mb
* 382 Mb
* 582 Mb
* `782 Mb`

You can get this information when running `docker images` - it'll be in the "SIZE" column.

In [28]:
# REPOSITORY                                                       TAG       IMAGE ID       CREATED         SIZE
# agrigorev/model-2024-hairstyle                                   v3        607850a2138e   2 days ago      782MB

A/ `782MB`

### Question 6

Now let's extend this docker image, install all the required libraries
and add the code for lambda.

You don't need to include the model in the image. It's already included. 
The name of the file with the model is `model-2024-hairstyle-v2.tflite` and it's 
in the current workdir in the image (see the Dockerfile above for the 
reference). 
The provided model requires the same preprocessing for images regarding target size and rescaling the value range than used in homework 8.

Now run the container locally.

Score this image: https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

What's the output from the model?

* 0.229
* `0.429`
* 0.629
* 0.829

In [47]:
import requests

url = 'http://localhost:8080/2015-03-31/functions/function/invocations'
data = { 'url': 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg' }

result = requests.post(url, json=data).json()

print(result)

{'pred': 0.42985355854034424}


A/ `0.4298`

### Publishing it to AWS

Now you can deploy your model to AWS!

* Publish your image to ECR
* Create a lambda function in AWS, use the ECR image
* Give it more RAM and increase the timeout 
* Test it
* Expose the lambda function using API Gateway

This is optional and not graded.

### Publishing to Docker hub

Just for the reference, this is how we published our image to Docker hub:

```bash
docker build -t model-2024-hairstyle -f homework.dockerfile .
docker tag model-2024-hairstyle:latest agrigorev/model-2024-hairstyle:v3
docker push agrigorev/model-2024-hairstyle:v3
```

(You don't need to execute this code)